In [1]:
import pandas as pd 
import datetime 
import json
import os

In [2]:
import great_expectations as ge
from ruamel import yaml

import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest

## Working with the 2020-2022 parquet data 

In [19]:
_df = pd.read_parquet('data/2020_22_liquor_month.parquet/')

In [31]:
_df.shape

(6257638, 23)

# Expectations & Suite built (method 4 in Step 3 methods in the offical document)

In [32]:
"""reference: 
1. https://docs.greatexpectations.io/docs/guides/miscellaneous/how_to_quickly_explore_expectations_in_a_notebook#2-load-some-data
2. https://github.com/StatCan/cpd_mlops_benchmarks/blob/master/notebooks/great_expectations/GreatExpect_EDA.ipynb

"""
def meta_notes(notes):
    
    meta_general =  {"author": "Yusa Li",
                    "source": "./cpd_mlops_benchmarks/yusa_price_expectations/price_expectation.ipynb"}
    extra_notes = [f"{key}: {value}" for key, value in meta_general.items()]
    
    notes =  {"notes": {
        "format": "markdown",
        "content": [
            notes
        ] + extra_notes
      }
    }
    notes.update(meta_general)
    
    return notes

In [33]:
# this is one approach to read in the data, can also read the in-memory pd dataframe 
# https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/in_memory/pandas

_df_ref = ge.read_parquet('./data/2020_22_liquor_month.parquet/partition_col=2020')


### 1. Table level summary

In [34]:
table_num_records = _df_ref.expect_table_row_count_to_be_between(
    min_value = 100_000, max_value = 10_000_000,
    result_format="SUMMARY",
    meta=meta_notes(notes="TEST_NOTE for now, will include more if needed"))

In [35]:
_df_ref.get_expectation_suite(discard_failed_expectations=False)

{
  "data_asset_type": "Dataset",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_table_row_count_to_be_between",
      "meta": {
        "notes": {
          "format": "markdown",
          "content": [
            "TEST_NOTE for now, will include more if needed",
            "author: Yusa Li",
            "source: ./cpd_mlops_benchmarks/yusa_price_expectations/price_expectation.ipynb"
          ]
        },
        "author": "Yusa Li",
        "source": "./cpd_mlops_benchmarks/yusa_price_expectations/price_expectation.ipynb"
      },
      "kwargs": {
        "min_value": 100000,
        "max_value": 10000000
      }
    }
  ],
  "meta": {
    "great_expectations_version": "0.15.14"
  },
  "expectation_suite_name": "default"
}

In [26]:
with open( "./great_expectations/expectations/first_liquor_expectation_file.json", "w") as my_file:
    my_file.write(
        json.dumps(_df_ref.get_expectation_suite().to_json_dict())
    )

### 2. Price max and min

In [36]:
sale_price_out_of_range = _df_ref.expect_column_values_to_be_between(
    column = 'Sale (Dollars)', 
    min_value = 1, 
    max_value = 2775
)

### ~~3. double sided z score, not ready yet, development needed~~

In [100]:
double_side_z = df_ge.expect_column_kl_divergence_to_be_less_than(
    column = 'Sale (Dollars)', 
    threshold = 3,
    result_format = 'BASIC'
)

AttributeError: 'PandasDataset' object has no attribute 'expect_column_value_z_scores_to_be_less_than'

### 4. KL test on distributio, 
**pasuse for now, Jul 20, 10am make sore the first teo expectations work for now**
https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/expectations/distributional_expectations.html#partition-object

https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/expectations/core/expect_column_kl_divergence_to_be_less_than/index.html

In [102]:
ge.get_context()

{
  "anonymous_usage_statistics": {
    "data_context_id": "6a410fdb-dd28-4471-8189-4886e4546f2d",
    "explicit_id": true,
    "explicit_url": false,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "enabled": true
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {
    "train_raw": {
      "class_name": "Datasource",
      "module_name": "great_expectations.datasource",
      "data_connectors": {
        "default_configured_data_connector_name": {
          "class_name": "ConfiguredAssetFilesystemDataConnector",
          "assets

In [106]:
# Continous partion calculation used for kl test
# https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/dataset/util/index.html#great_expectations.dataset.util.build_continuous_partition_object

ge.dataset.util.build_continuous_partition_object(df_ge, column = 'Sale (Dollars)')

{'bins': [1.3,
  2.627457930180764,
  3.9549158603615284,
  5.282373790542292,
  6.609831720723056,
  7.93728965090382,
  9.264747581084585,
  10.59220551126535,
  11.919663441446113,
  13.247121371626877,
  14.574579301807642,
  15.902037231988405,
  17.22949516216917,
  18.556953092349932,
  19.8844110225307,
  21.211868952711463,
  22.539326882892226,
  23.86678481307299,
  25.194242743253753,
  26.52170067343452,
  27.849158603615283,
  29.176616533796047,
  30.50407446397681,
  31.831532394157573,
  33.15899032433833,
  34.4864482545191,
  35.81390618469986,
  37.14136411488063,
  38.468822045061394,
  39.79627997524216,
  41.12373790542292,
  42.451195835603684,
  43.77865376578445,
  45.10611169596521,
  46.433569626145974,
  47.76102755632674,
  49.0884854865075,
  50.415943416688265,
  51.743401346869035,
  53.0708592770498,
  54.39831720723056,
  55.725775137411325,
  57.05323306759209,
  58.38069099777285,
  59.708148927953616,
  61.03560685813438,
  62.36306478831514,
  63.

### Save suite

In [37]:
## show the expection suite 
_df_ref.get_expectation_suite(discard_failed_expectations=False)

{
  "data_asset_type": "Dataset",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_table_row_count_to_be_between",
      "meta": {
        "notes": {
          "format": "markdown",
          "content": [
            "TEST_NOTE for now, will include more if needed",
            "author: Yusa Li",
            "source: ./cpd_mlops_benchmarks/yusa_price_expectations/price_expectation.ipynb"
          ]
        },
        "author": "Yusa Li",
        "source": "./cpd_mlops_benchmarks/yusa_price_expectations/price_expectation.ipynb"
      },
      "kwargs": {
        "min_value": 100000,
        "max_value": 10000000
      }
    },
    {
      "expectation_type": "expect_column_values_to_be_between",
      "meta": {},
      "kwargs": {
        "column": "Sale (Dollars)",
        "min_value": 1,
        "max_value": 2775
      }
    }
  ],
  "meta": {
    "great_expectations_version": "0.15.14"
  },
  "expectation_suite_name": "default"
}

In [44]:
_df_ref.validate()['success']

False

In [98]:
_df_ref.expectation_suite_name = "unit_price_suite"
_df_ref.save_expectation_suite("unit_price_suite.json", discard_failed_expectations=False)

In [46]:
_df_ref.expectation_suite_name = "liquor_suite_aug_11"
_df_ref.save_expectation_suite("./great_expectations/expectations/liquor_suite_aug_11.json", discard_failed_expectations=False)

# 2. Data source setup

READ data from in memory df
https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_create_a_batch_of_data_from_an_in_memory_spark_or_pandas_dataframe

## 2.1) Attempt on example_suite and example_datasource

In [ ]:
context = ge.get_context()

In [ ]:
datasource_config = {
    "name": "example_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "module_name": "great_expectations.datasource.data_connector",
            "batch_identifiers": ["default_identifier_name"],
        },
    },
}

In [ ]:
context.test_yaml_config(yaml.dump(datasource_config))

'''Note: Since the Datasource does not have data passed-in until later, 
the output will show that no data_asset_names are currently available. 
This is to be expected.
'''

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



/Users/yusali/anaconda3/envs/mlops/lib/python3.10/site-packages/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


In [ ]:
context.add_datasource(**datasource_config)

/Users/yusali/anaconda3/envs/mlops/lib/python3.10/site-packages/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


In [ ]:
batch_request = RuntimeBatchRequest(
    datasource_name="example_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="2019_base_data",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"batch_data": df_19},  # df is your dataframe
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

In [ ]:
context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request, 
    expectation_suite_name="test_suite"
)
# print(validator.head())

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 171.38it/s]

  Invoice/Item Number        Date  Store Number  \
0     INV-18212400042  2019-03-19          2513   
1     INV-19392400209  2019-05-15          5068   
2     INV-19405700109  2019-05-16          2588   
3     INV-19379800083  2019-05-15          2647   
4     INV-19363000015  2019-05-14          5202   

                               Store Name               Address          City  \
0        Hy-Vee Food Store #2 / Iowa City        812  S 1st Ave     Iowa City   
1             Tobacco Shop / Arnolds Park      612 S Highway 71  Arnolds Park   
2  Hy-Vee Food and Drug #6 / Cedar Rapids  4035 Mt Vernon Rd SE  Cedar Rapids   
3                Hy-Vee #7 / Cedar Rapids      5050 Edgewood Rd  Cedar Rapids   
4    Casey's General Store #2900 / Gilber        115, 25th  Ave  Gilbertville   

  Zip Code                        Store Location  County Number      County  \
0    52240                                   NaN           52.0     JOHNSON   
1    51331  POINT (-95.14027400000002 43.361577)

In [ ]:
validator.get_metric()

NameError: name 'validator' is not defined

# 4. Validattion

In [ ]:
# Create second checkpoint on yellow_tripdata_sample_2019-02.csv
# <snippet>
yaml_config = f"""
name: getting_started_checkpoint
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  # - batch_request:
      # datasource_name: getting_started_datasource
      # data_connector_name: default_inferred_data_connector_name
      # data_asset_name: yellow_tripdata_sample_2019-02.csv
    
  - batch_request : RuntimeBatchRequest(
    datasource_name : "example_datasource",
    data_connector_name:"default_runtime_data_connector_name",
    data_asset_name:"2019_base_data",  # This can be anything that identifies this data_asset for you
    runtime_parameters:{"batch_data": df_ref},  # df is your dataframe
    batch_identifiers:{"default_identifier_name": "default_identifier"},
)
"""

## 2.2) Liquor Data source attempt

In [45]:
context = ge.get_context()

/Users/yusali/anaconda3/envs/mlops/lib/python3.10/site-packages/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


In [24]:
context.get_validator

<bound method BaseDataContext.get_validator of {
  "anonymous_usage_statistics": {
    "explicit_id": true,
    "data_context_id": "9d3aedba-5ea1-45ad-8470-fabefdf74fb1",
    "enabled": true,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_url": false
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {
    "example_datasource": {
      "data_connectors": {
        "default_runtime_data_connector_name": {
          "batch_identifiers": [
            "defa

In [25]:
datasource_yaml = f"""
name: liquor_datasource_ref
class_name: Datasource
module_name: great_expectations.datasource
execution_engine:
  module_name: great_expectations.execution_engine
  class_name: PandasExecutionEngine
data_connectors:
    default_runtime_data_connector_name:
        class_name: RuntimeDataConnector
        batch_identifiers:
            - default_identifier_name
"""

In [26]:
context.test_yaml_config(datasource_yaml)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [27]:
context.add_datasource(**yaml.load(datasource_yaml))

/var/folders/0x/3ms0bnbn6196rf0m4bwzwdrm0000gn/T/ipykernel_15337/2045274942.py:1: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  context.add_datasource(**yaml.load(datasource_yaml))


In [ ]:
batch_request = RuntimeBatchRequest(
    datasource_name="liquor_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="liquor_data",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"batch_data": df_to_validate},  # Pass your DataFrame here.
    batch_identifiers={"default_identifier_name": "liquor_data_all"},
)

In [28]:
batch_request = RuntimeBatchRequest(
    datasource_name="liquor_datasource_ref",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="liquor_data_ref",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"batch_data": df_ref},  # Pass your DataFrame here.
    batch_identifiers={"default_identifier_name": "liquor_data_ref"},
)

In [29]:
# context.create_expectation_suite(
#     expectation_suite_name="liquor_suite_aug_11", overwrite_existing=True
# )
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="liquor_suite_aug_11"
)
print(validator.head())

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 316.07it/s]

  Invoice/Item Number  Store Number                        Store Name  \
0     INV-24292600043          4404            KUM & GO #292 / Ankeny   
1     INV-24306300051          5257                 MAD Ave Quik Shop   
2     INV-24290400075          2502  Hy-Vee Wine and Spirits / Ankeny   
3     INV-24312800016          5709                         JW Liquor   
4     INV-24294900180          2647          Hy-Vee #7 / Cedar Rapids   

                            Address          City  \
0                1825 N Ankeny Blvd        Ankeny   
1                  405, Madison Ave       Ottumwa   
2             410 North Ankeny Blvd        Ankeny   
3  4518 Mortonsen Street Suite #109          Ames   
4                  5050 Edgewood Rd  Cedar Rapids   

                 Store Location  County Number   County   Category  \
0                          None           77.0     POLK  1012100.0   
1                          None           90.0  WAPELLO  1031100.0   
2                          None 

In [30]:
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler
profiler = UserConfigurableProfiler(profile_dataset=validator)

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 792.28it/s] 


In [41]:
# suite = context.get_expectation_suite(
#     expectation_suite_name="liquor_suite_aug_11"
# )

suite = profiler.build_suite()

Profiling: 100%|██████████| 23/23 [01:12<00:00,  3.16s/it, Column=partition_col]

Creating an expectation suite with the following expectations:

Table-Level Expectations
expect_table_columns_to_match_ordered_list
expect_table_row_count_to_be_between

Expectations by Column
Column Name: Address | Column Data Type: STRING | Cardinality: MANY
expect_column_proportion_of_unique_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: Bottle Volume (ml) | Column Data Type: INT | Cardinality: FEW
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: Bottles Sold | Column Data Type: INT | Cardinality: MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_med

In [47]:
expectation_suite_name = 'liquor_suite_aug_11'

In [43]:
from great_expectations.checkpoint.checkpoint import SimpleCheckpoint

# Review and save our Expectation Suite 
print(validator.get_expectation_suite(discard_failed_expectations=False))
validator.save_expectation_suite(discard_failed_expectations=False)

# Set up and run a Simple Checkpoint for ad hoc validation of our data 
checkpoint_config = {
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name,
        }
    ],
}
checkpoint = SimpleCheckpoint(
    f"{validator.active_batch_definition.data_asset_name}_{expectation_suite_name}", context, **checkpoint_config
)
checkpoint_result = checkpoint.run()

# Build Data Docs
context.build_data_docs()

# Get the only validation_result_identifier from our SimpleCheckpoint run, and open Data Docs to that page
validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0] 
context.open_data_docs(resource_identifier=validation_result_identifier)

{
  "data_asset_type": null,
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_table_columns_to_match_ordered_list",
      "meta": {},
      "kwargs": {
        "column_list": [
          "Invoice/Item Number",
          "Store Number",
          "Store Name",
          "Address",
          "City",
          "Store Location",
          "County Number",
          "County",
          "Category",
          "Category Name",
          "Vendor Number",
          "Vendor Name",
          "Item Number",
          "Item Description",
          "Pack",
          "Bottle Volume (ml)",
          "State Bottle Cost",
          "State Bottle Retail",
          "Bottles Sold",
          "Sale (Dollars)",
          "Volume Sold (Liters)",
          "Volume Sold (Gallons)",
          "partition_col"
        ]
      }
    },
    {
      "expectation_type": "expect_table_row_count_to_be_between",
      "meta": {},
      "kwargs": {
        "min_value": 2614365,
        "m

ValueError: Error: batch_data found in batch_request -- only primitive types are allowed as Checkpoint constructor arguments.


# 4. Validation and checkpoint with new data source 